In [ ]:
# this notebook is going to test for randomly sampled locally equivalent gate, whether the basis gate set is sufficient to reconstruct the unitary with smushed gates
# we are going to test sqiswap, cnot, and syc as the 3 basis gates realized from gainconversion, CR, and fsim hamiltonians

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

%matplotlib widget


import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

%matplotlib widget

import sys
sys.path.append("../../../")
import numpy as np

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import random_unitary
from src.basisv2 import CircuitTemplateV2
from src.utils.custom_gates import ConversionGainSmushGate, ConversionGainGate
from src.sampler import GateSample
from qiskit.extensions import UnitaryGate
from qiskit.quantum_info import Operator
from src.optimizer import TemplateOptimizer
from src.cost_function import BasicCost, SquareCost

In [ ]:
t= 1
timesteps = 10
duration_1q = t/timesteps
bounds_1q = 2*np.pi

# fixing these to be iterated over
gc, gg = (1*np.pi/4, 0*np.pi/4)

varg_offset = 0 #set to 4 if want to use phase, and change 0s to vargs in pp2 constructor below
pp2 =lambda *vargs: ConversionGainSmushGate(0,0 , gc, gg, vargs[varg_offset:varg_offset+round(t/duration_1q)], vargs[varg_offset+round(t/duration_1q):], t_el=t)
basis = CircuitTemplateV2(n_qubits=2, base_gates = [pp2], edge_params=[[(0,1)]], vz_only=True, param_vec_expand=[varg_offset,round(t/duration_1q),round(t/duration_1q)])
basis_str = "CG2Q"

# varg_offset = 2
# pp3 =lambda *vargs: ConversionGainSmushGate(vargs[0], vargs[1], gc, gg, vargs[varg_offset:varg_offset+round(t/duration_1q)], vargs[varg_offset+round(t/duration_1q):], t_el=t)
# basis = CircuitTemplateV2(n_qubits=2, base_gates = [pp3], edge_params=[[(0,1)]], vz_only=True, param_vec_expand=[varg_offset,round(t/duration_1q),round(t/duration_1q)])
# basis_str = "CG2Q+P"

basis.build(1)
basis.spanning_range = range(1,2)

#for all smush gates make bounds
for el in basis.circuit.parameters:
    s_el = str(el)
    if 'Q' in s_el:
        basis.add_bound(s_el, bounds_1q, 0)

# # manually set the gc, gg bounds
# for el in basis.circuit.parameters:
#     s_el = str(el)
#     if s_el in ["Q0", "Q1", "Q22", "Q23", "Q44", "Q45"]:
#         basis.add_bound(s_el, .5, 0)
#         #basis.add_bound(s_el, 0.5*np.pi, 0) #XXX when using riswapgate should be 0.5, but when using ConversionGain should be np.pi/2 !!! BAD BAD BAD


# construct the locally variant target

qc = QuantumCircuit(2)
# qc.append(random_unitary(2), [0])
# qc.append(random_unitary(2), [1])
g = ConversionGainGate(0, 0, gc, gg, t_el=t)
qc.append(g, [0,1])
# qc.append(random_unitary(2), [0])
# qc.append(random_unitary(2), [1])
# qc.rx(-np.pi/2, 0)
# qc.rx(-np.pi/2, 1)
# qc.draw('mpl');
gate_data = Operator(qc).data

from src.utils.custom_gates import CanonicalGate
sampler = GateSample(CanonicalGate(np.pi / 4, np.pi / 8, np.pi/8))

from qiskit.circuit.library import CPhaseGate
gate_data = CPhaseGate(theta= 1.5* np.pi).to_matrix()
sampler = GateSample(UnitaryGate(gate_data))
s = [s for s in sampler][0]

objective1 = SquareCost()
optimizer3 = TemplateOptimizer(basis=basis, objective=objective1, use_callback=True, override_fail=True, success_threshold = 1e-7, training_restarts=3)
basis.circuit.draw(output='mpl');

In [ ]:
_ret3 = optimizer3.approximate_from_distribution(sampler)
from src.utils.visualize import optimizer_training_plot
optimizer_training_plot(_ret3[0], _ret3[1])

In [ ]:
# _ret3 = optimizer3.approximate_target_U(s)
#ret3[2] gets the target_data list, [0] returns the first element over the sampler distribution (we only put one in)
basis.reconstruct(_ret3[2][0]).draw()